#TBrain E.SUN AI Open Competition Fall 2019 15th place(top1%) Feature Engineering
#--------------------------------------------------------------------------#
#Team:菜雞互啄
#Members:Alexi,Pat,Michael,Ming-Xiang,Ethan
#F1 score prediction competition: 15th of 1366 teams
#Creativity presentation competition: 2nd
#Competition Link: https://tbrain.trendmicro.com.tw/Competitions/Details/10
#--------------------------------------------------------------------------#

In [1]:
%reset -f
import pandas as pd
import numpy as np
import copy
from collections import deque
from pandas.core.frame import DataFrame

In [2]:
#Sample Data
df = pd.DataFrame(np.random.randint(0,1000,size=(10000000, 1)),columns = ["time"])
df['GroupKey'] = np.random.choice(a=[10, 20, 30,40,50,60,70,80,90,100],size=10000000,p=[0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1])
df['Conam'] = np.random.randint(300,800,size=(10000000, 1)) #simulate CONsumer AMount
df['ID'] = df.index

#sort data
#Notice:You need to sort data by  yourself for the purpose of Diff_func can get correct mode feature(s)
df = df.sort_values(["GroupKey","time"])

In [3]:
def Diff_func(df,JoinKey,GroupKey,Var):            
    tmp = copy.deepcopy(df)
    tmp_list = [list(tmp[JoinKey]),
                list(tmp[GroupKey]),
                list(tmp[Var]),
                [0]*tmp.shape[0], #rolling mean
                [0]*tmp.shape[0], #difference between consumer amount and rolling mean
                [0]*tmp.shape[0] #diff log
                ]
    data=[]
    for j in range(len(tmp_list[0])):
        data.append([tmp_list[i][j] for i in range(len(tmp_list))])            
    
    GroupKey_bool = None
    for i in range(len(data)):
        if data[i][1]!=GroupKey_bool:
            GroupKey_bool=data[i][1]
            sumvalue=0;count=0
            sumvalue=data[i][2]
        else:
            count+=1
            data[i][3]=sumvalue/count
            data[i][4]=data[i][2]-data[i][3]#diff
            data[i][5]=np.log10(abs(data[i][4]))
            sumvalue+=data[i][2]
            
    data = DataFrame(data)
    data.columns = [str(JoinKey),str(GroupKey),str(Var),"rolling_mean","diff_origin","diff_log"]        
    globals()[Var + "_DifferenceFrame" ] = data
    print("Done: "+Var+"_DifferenceFrame")        

In [4]:
import time
tStart = time.time()

Diff_func(df,'ID','GroupKey','Conam')
tEnd = time.time()
print ("It cost %f sec" % (tEnd - tStart))

Done: Conam_DifferenceFrame
It cost 45.670646 sec


In [5]:
Conam_DifferenceFrame.head(10)

,ID,GroupKey,Conam,rolling_mean,diff_origin,diff_log
0,10275,10,718,0.000000,0.000000,0.000000
1,22628,10,387,718.000000,-331.000000,2.519828
2,26080,10,516,552.500000,-36.500000,1.562293
3,27710,10,655,540.333333,114.666667,2.059437
4,34578,10,546,569.000000,-23.000000,1.361728
5,39363,10,362,564.400000,-202.400000,2.306211
6,45625,10,687,530.666667,156.333333,2.194052
7,52710,10,320,553.000000,-233.000000,2.367356
8,69211,10,364,523.875000,-159.875000,2.203781
9,74930,10,775,506.111111,268.888889,2.429573
